I made Q-learning starter notebook.

It's first time to try reinforcement learning, so if you find mistake, please tell me by comment. Also, I'm glad if you share me opinion to make good state key by comment.


Therer are two improvements from the code in the beta version.
1. This is Strategy base. Before that, agent select each action by observing the number of kore. It didn't work well maybe because it has to spend a large amount of time learning.
2. Learn by long time reward. After launching ships, it require dozens of steps to get kore.

In [ ]:
from kaggle_environments import make

env = make("kore_fleets", debug=True)
print(env.name, env.version)

In [ ]:
from kaggle_environments.envs.kore_fleets.helpers import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random
import seaborn as sns
from tqdm import tqdm
import itertools
import pickle

In [ ]:
# Decide nvm
# 0~50 -> 0.97
# 50~ -> 0.98
# 100~ -> 0.99

repeat_size=1000
x = [i for i in range(repeat_size)]
y = [min(0.977 + np.log(i*0.02+1)*0.005, 0.99) for i in range(repeat_size)]
plt.plot(x, y)

In [ ]:
plt.plot(x[:100], y[:100])

# Start Q-learning
I refer to this article [kaggleで強化学習をやってみた](https://yukoishizaki.hatenablog.com/entry/2020/04/05/202935) to make QTable and QLearningAgent.

In [ ]:
action_ratios = [
    (5, 2, 2, 2),
    (2, 5, 2, 2),
    (2, 2, 5, 2),
    (2, 2, 2, 5),
    (1, 7, 4, 1),
    (1, 4, 7, 1),
    (1, 5, 1, 1),
    (3, 5, 0, 0),
    (1, 5, 5, 0),
    (1, 5, 5, 5),
    (1, 1, 5, 1),
    (1, 4, 4, 8),
    (1, 5, 2, 5),
    (3, 5, 2, 5),
    (1, 4, 8, 4),
    (8, 5, 1, 1),
    (3, 3, 3, 5),
    (3, 3, 5, 3),
    (3, 5, 3, 3),
    (5, 3, 3, 3)
]

In [ ]:
ship_max=3000
x = [i for i in range(ship_max)]
y = [np.log(i*0.01+1)*10 for i in range(ship_max)]
plt.plot(x, y)


In [ ]:
class QTable():
    def __init__(self, ratios):
        self.Q = {} # Qテーブル
        self.ratios = ratios
    
    def get_state_key(self, state):
        board = Board(state, env.configuration)
        me=board.current_player
        me_obser = me._observation
        
        turn = board.step
        kore_left = me.kore
        
        ship_number = 0
        for value in me_obser[1].values():
            ship_number += value[1]
        
        fleet_number = 0
        for value in me_obser[2].values():
            fleet_number += value[2]

        return f'{int(turn/20)}'
        
    def get_q_values(self, state):
        # 状態に対して、全actionのQ値の配列を出力
        state_key = self.get_state_key(state)
        if state_key not in self.Q.keys():
            self.Q[state_key] = [0] * len(self.ratios)
        return self.Q[state_key]
    
    def update(self, state, ratio_index, add_q):
        # Q値を更新
        state_key = self.get_state_key(state)
        self.Q[state_key] = [q + add_q if idx == ratio_index else q for idx, q in enumerate(self.Q[state_key])]

In [ ]:
# Helper
def pos_to_index(pos):
    x = pos[0]
    y = pos[1]
    x = (x + 21) if x < 0 else x
    x = (x - 21) if x > 20 else x
    y = (y + 21) if y < 0 else y
    y = (y - 21) if y > 20 else y

    return x * 21 + y

# Get straight score
def get_straight_score(pos, kore, length, direction):
    score = 0
    for i in range(length):
        pos += direction.to_point()
        score += kore[pos_to_index(pos)]
    return score

def get_max_straight_plan(pos, kore, length):
    scores = []
    for i in range(4):
        direction = Direction.from_index(i)
        scores.append(get_straight_score(pos, kore, length, direction))
    scores = sorted(scores, reverse=True)
#     direction_index = scores.index(scores[np.random.randint(2)])
    direction_index = scores.index(scores[0])
    
    direction = Direction.from_index(direction_index)
    flight_plan = direction.to_char()
    flight_plan += str(length)
    flight_plan += direction.opposite().to_char()
    return flight_plan


# Get maximum cycle
def get_max_cycle_plan(pos, kore, length, direct_i):
    direction = Direction.from_index(direct_i) 
    each_lengths = [i for i in range(1,length)]
    each_lengths = [i for i in itertools.product(each_lengths, repeat=2)]
    each_lengths = [i + tuple([i[0], i[1]]) for i in each_lengths]
    
    # Get max score
    scores = []
    for rotate_i in range(2):
        for each_length in each_lengths:
            each_pos = pos
            each_direction = direction
            score = 0
            for direct_i in range(4):
                for i in range(each_length[direct_i]):
                    each_pos += each_direction.to_point()
                    score += kore[pos_to_index(each_pos)]
                if rotate_i == 0:
                    each_direction = each_direction.rotate_right()
                else:
                    each_direction = each_direction.rotate_left()
            scores.append(score)
    
    # Get flight plan
    rotate_right = True
    best_index = scores.index(max(scores))
    if best_index >= len(each_lengths):
        best_index -= len(each_lengths)
        rotate_right = False
        
    best_each_length = each_lengths[best_index]
    each_direction = direction
    flight_plan = ""
    for i in range(4):
        flight_plan += each_direction.to_char()
        
        length = best_each_length[i] - 1
        if i != 3 and length > 0:
            flight_plan += str(length)
        if rotate_right:
            each_direction = each_direction.rotate_right()
        else:
            each_direction = each_direction.rotate_left()
    return flight_plan

def get_closest_enemy_shipyard(board, position, me):
    min_dist = 1000000
    enemy_shipyard = None
    for shipyard in board.shipyards.values():
        if shipyard.player_id == me.id:
            continue
        dist = position.distance_to(shipyard.position, board.configuration.size)
        if dist < min_dist:
            min_dist = dist
            enemy_shipyard = shipyard
    return enemy_shipyard

In [ ]:
def shipyard_action(action_number, board, shipyard):
    me = board.current_player
    kore_left = me.kore
    pos = shipyard.position
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    observation = board.observation
    kore = observation['kore']

    if action_number == 0:
        # Spawn
        if int(kore_left / spawn_cost) == 0:
            return None
        spawn_max = shipyard.max_spawn
        return ShipyardAction.spawn_ships(min(spawn_max, int(kore_left / spawn_cost)))
    
    elif action_number == 1:
        # Cycle
        if shipyard.ship_count < 21:
            return None
        length = random.randint(min(int(turn/20)+2, 6), 9)
        direct_i = random.randint(0, 3)
        flight_plan = get_max_cycle_plan(pos, kore, length, direct_i)
        ship_number = 21
        return ShipyardAction.launch_fleet_with_flight_plan(ship_number, flight_plan)
    
    elif action_number == 2:
        # Invade
        if shipyard.ship_count < 100 + 21:
            return None

        closest_enemy_shipyard = get_closest_enemy_shipyard(board, pos, me)
        if not closest_enemy_shipyard:
            return None
        enemy_pos = closest_enemy_shipyard.position
        my_pos = pos
        flight_plan = "N" if enemy_pos.y > my_pos.y else "S"
        flight_plan += str(abs(enemy_pos.y - my_pos.y) - 1)
        flight_plan += "W" if enemy_pos.x < my_pos.x else "E"
        if (abs(enemy_pos.y - my_pos.y) - 1) < 0:
            return None
        if not all([c in "NESWC0123456789" for c in flight_plan]):
            return None

        ship_number = 100
        return ShipyardAction.launch_fleet_with_flight_plan(ship_number, flight_plan)

    elif action_number == 3:
        # Make shipyard
        if shipyard.ship_count < 75 + 21:
            return None

        length = random.randint(3, 8)
        direct_i = random.randint(0, 3)
        flight_plan = get_max_cycle_plan(pos, kore, length, direct_i)
        flight_plan = flight_plan[:-3]
        flight_plan += 'C'
        ship_number = max(75, int(shipyard.ship_count*(random.random()/2 + 0.5)))
        return ShipyardAction.launch_fleet_with_flight_plan(ship_number, flight_plan)
    

In [ ]:
env = make("kore_fleets", debug=True)
# # Balanced without attack
# trainer = env.train([None, "./balanced.py"])

# My best rule based model with attack
trainer = env.train([None, "../input/kore2022bots/builder (46).py"])

class QLearningAgent():
    def __init__(self, env, epsilon=0.99):
        self.env = env
        self.ratios = action_ratios.copy()
        self.q_table = QTable(self.ratios)
        self.epsilon = epsilon
        self.reward_log = []
        self.current_policy = -1
        
    def policy(self, state, shipyard, kore_left):
        if np.random.random() < self.epsilon:
            # epsilonの割合で、ランダムにactionを選択する
            return random.randint(0, len(self.ratios)-1)
        else:
            # ゲーム上選択可能で、Q値が最大なactionを選択する
            q_values = self.q_table.get_q_values(state)
            return int(np.argmax(q_values))
        
    def custom_reward(self, reward, done):
        if done:
            if reward == 1: # 勝ち
                return 20
            elif reward == 0: # 負け
                return -20
            else: # 引き分け
                return 10
        else:
            return -0.05 # 勝負がついてない
        
    def learn(self, trainer, episode_cnt=100, gamma=0.6, 
              learn_rate=0.3, epsilon_decay_rate=0.99, min_epsilon=0.1):
        for episode in tqdm(range(episode_cnt)):
            # ゲーム環境リセット
            state = trainer.reset() 
            # epsilonを徐々に小さくする
            self.epsilon = max(min_epsilon, self.epsilon * epsilon_decay_rate) 
            while not env.done:
                # Update q to each shipyard.
                board = Board(state, self.env.configuration)
                spawn_cost = board.configuration.spawn_cost

                me = board.current_player
                turn = board.step
                kore_left = me.kore
                
                each_selections = []
                # loop through all shipyards you control
                for shipyard in me.shipyards:
                    spawn_max = shipyard.max_spawn
                    if int(kore_left / spawn_cost) > 0:
                        shipyard.next_action = ShipyardAction.spawn_ships(min(spawn_max, int(kore_left / spawn_cost)))
                    
                    if turn % 10 == 0:
                        ratio_idx = self.policy(state, shipyard, kore_left)
                        self.current_policy = ratio_idx
                    else:
                        ratio_idx = self.current_policy
                    ratios = self.ratios[ratio_idx]
                    each_selections.append(ratio_idx)
                    
                    ratios = list(itertools.chain.from_iterable([[i]*n for i, n in enumerate(ratios)]))
                    action_number = random.choice(ratios)
                    action = shipyard_action(action_number, board, shipyard)
                    if action != None:
                        shipyard.next_action = action
                next_state, reward, done, info = trainer.step(me.next_actions)
                # reward = self.custom_reward(reward, done)
                
                if turn % 10 == 9:
                    reward = kore_left
                    action_value_rate = 1.2
                    for value in me._observation[2].values():
                        reward += (value[2] * (10 * action_value_rate))
                    reward += (len(me.shipyards) * (50 * action_value_rate))
                    gain = reward + gamma * max(self.q_table.get_q_values(next_state))
                    
                    estimate = self.q_table.get_q_values(state)[self.current_policy]
                    self.q_table.update(state, self.current_policy, learn_rate * (gain - estimate))
                    
                state = next_state
            self.reward_log.append(reward)

In [ ]:
episode_cnt=1500
epsilon_decay_rate=0.9995
x = [i for i in range(episode_cnt)]
y = [max(0.99*(epsilon_decay_rate**i), 0.1) for i in range(episode_cnt)]
plt.plot(x, y)

In [ ]:
# Learning
qa = QLearningAgent(env)
qa.learn(trainer, episode_cnt=episode_cnt, epsilon_decay_rate=epsilon_decay_rate)
# qa.learn(trainer, episode_cnt=1000)

In [ ]:
# Mean of transition of reward
sns.set(style='darkgrid')
pd.DataFrame({'Average Reward': qa.reward_log}).rolling(10).mean().plot(figsize=(10,5))
plt.show()

In [ ]:
sns.set(style='darkgrid')
pd.DataFrame({'Average Reward': qa.reward_log}).plot(figsize=(10,5))
plt.show()

In [ ]:
tmp_dict_q_table = qa.q_table.Q.copy()
dict_q_table = dict()

# Replace best q-value action ratio
for k in tmp_dict_q_table:
    if np.count_nonzero(tmp_dict_q_table[k]) > 0:
        dict_q_table[k] = np.argsort(tmp_dict_q_table[k]).argsort().tolist()[0]

In [ ]:
my_agent = '''
from kaggle_environments.envs.kore_fleets.helpers import *
import numpy as np
import itertools
import random
# Helper
def pos_to_index(pos):
    x = pos[0]
    y = pos[1]
    x = (x + 21) if x < 0 else x
    x = (x - 21) if x > 20 else x
    y = (y + 21) if y < 0 else y
    y = (y - 21) if y > 20 else y

    return x * 21 + y

# Get straight score
def get_straight_score(pos, kore, length, direction):
    score = 0
    for i in range(length):
        pos += direction.to_point()
        score += kore[pos_to_index(pos)]
    return score

def get_max_straight_plan(pos, kore, length):
    scores = []
    for i in range(4):
        direction = Direction.from_index(i)
        scores.append(get_straight_score(pos, kore, length, direction))
    scores = sorted(scores, reverse=True)
    direction_index = scores.index(scores[0])

    direction = Direction.from_index(direction_index)
    flight_plan = direction.to_char()
    flight_plan += str(length)
    flight_plan += direction.opposite().to_char()
    return flight_plan


# Get maximum cycle
def get_max_cycle_plan(pos, kore, length, direct_i):
    direction = Direction.from_index(direct_i) 
    each_lengths = [i for i in range(1,length)]
    each_lengths = [i for i in itertools.product(each_lengths, repeat=2)]
    each_lengths = [i + tuple([i[0], i[1]]) for i in each_lengths]
    
    # Get max score
    scores = []
    for rotate_i in range(2):
        for each_length in each_lengths:
            each_pos = pos
            each_direction = direction
            score = 0
            for direct_i in range(4):
                for i in range(each_length[direct_i]):
                    each_pos += each_direction.to_point()
                    score += kore[pos_to_index(each_pos)]
                if rotate_i == 0:
                    each_direction = each_direction.rotate_right()
                else:
                    each_direction = each_direction.rotate_left()
            scores.append(score)
    
    # Get flight plan
    rotate_right = True
    best_index = scores.index(max(scores))
    if best_index >= len(each_lengths):
        best_index -= len(each_lengths)
        rotate_right = False
        
    best_each_length = each_lengths[best_index]
    each_direction = direction
    flight_plan = ""
    for i in range(4):
        flight_plan += each_direction.to_char()
        
        length = best_each_length[i] - 1
        if i != 3 and length > 0:
            flight_plan += str(length)
        if rotate_right:
            each_direction = each_direction.rotate_right()
        else:
            each_direction = each_direction.rotate_left()
    return flight_plan


def get_closest_enemy_shipyard(board, position, me):
    min_dist = 1000000
    enemy_shipyard = None
    for shipyard in board.shipyards.values():
        if shipyard.player_id == me.id:
            continue
        dist = position.distance_to(shipyard.position, board.configuration.size)
        if dist < min_dist:
            min_dist = dist
            enemy_shipyard = shipyard
    return enemy_shipyard


def shipyard_action(action_number, board, shipyard):
    me = board.current_player
    kore_left = me.kore
    pos = shipyard.position
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    observation = board.observation
    kore = observation['kore']

    if action_number == 0:
        # Spawn
        if int(kore_left / spawn_cost) == 0:
            return None
        spawn_max = shipyard.max_spawn
        return ShipyardAction.spawn_ships(min(spawn_max, int(kore_left / spawn_cost)))
    
    elif action_number == 1:
        # Cycle
        if shipyard.ship_count < 21:
            return None
        length = random.randint(min(int(turn/20)+2, 6), 9)
        direct_i = random.randint(0, 3)
        flight_plan = get_max_cycle_plan(pos, kore, length, direct_i)
        ship_number = 21
        return ShipyardAction.launch_fleet_with_flight_plan(ship_number, flight_plan)
    
    elif action_number == 2:
        # Invade
        if shipyard.ship_count < 100 + 21:
            return None

        closest_enemy_shipyard = get_closest_enemy_shipyard(board, pos, me)
        if not closest_enemy_shipyard:
            return None
        enemy_pos = closest_enemy_shipyard.position
        my_pos = pos
        flight_plan = "N" if enemy_pos.y > my_pos.y else "S"
        flight_plan += str(abs(enemy_pos.y - my_pos.y) - 1)
        flight_plan += "W" if enemy_pos.x < my_pos.x else "E"
        if (abs(enemy_pos.y - my_pos.y) - 1) < 0:
            return None
        if not all([c in "NESWC0123456789" for c in flight_plan]):
            return None

        ship_number = 100
        return ShipyardAction.launch_fleet_with_flight_plan(ship_number, flight_plan)

    elif action_number == 3:
        # Make shipyard
        if shipyard.ship_count < 75 + 21:
            return None

        length = random.randint(3, 8)
        direct_i = random.randint(0, 3)
        flight_plan = get_max_cycle_plan(pos, kore, length, direct_i)
        flight_plan = flight_plan[:-3]
        flight_plan += 'C'
        ship_number = max(75, int(shipyard.ship_count*(random.random()/2 + 0.5)))
        return ShipyardAction.launch_fleet_with_flight_plan(ship_number, flight_plan)

action_ratios = [
    (5, 2, 2, 2),
    (2, 5, 2, 2),
    (2, 2, 5, 2),
    (2, 2, 2, 5),
    (1, 7, 4, 1),
    (1, 4, 7, 1),
    (1, 5, 1, 1),
    (3, 5, 0, 0),
    (1, 5, 5, 0),
    (1, 5, 5, 5),
    (1, 1, 5, 1),
    (1, 4, 4, 8),
    (1, 5, 2, 5),
    (3, 5, 2, 5),
    (1, 4, 8, 4),
    (8, 5, 1, 1),
    (3, 3, 3, 5),
    (3, 3, 5, 3),
    (3, 5, 3, 3),
    (5, 3, 3, 3)
]

def agent(obs, config):
    # Convert table to string for using as python file.
    q_table = ''' \
    + str(dict_q_table).replace(' ', '') \
    + '''
    
    board = Board(obs, config)
    me=board.current_player
    me_obser = me._observation

    spawn_cost = board.configuration.spawn_cost
    turn = board.step
    kore_left = me.kore
    
    observation = board.observation
    kore = observation['kore']

    ship_number = 0
    for value in me_obser[1].values():
        ship_number += value[1]
    fleet_number = 0
    for value in me_obser[2].values():
        fleet_number += value[2]
        
    state_key = f'{int(turn/20)}'

    each_selections = []
    # loop through all shipyards you control
    for shipyard in me.shipyards:
        spawn_max = shipyard.max_spawn
        if int(kore_left / spawn_cost) > 0:
            shipyard.next_action = ShipyardAction.spawn_ships(min(spawn_max, int(kore_left / spawn_cost)))

        if state_key not in q_table.keys():
            ratio_idx = random.randint(0, len(action_ratios)-1)
        else:
            ratio_idx = q_table[state_key]
        ratios = action_ratios[ratio_idx]
        ratios = list(itertools.chain.from_iterable([[i]*n for i, n in enumerate(ratios)]))
        each_selections.append(ratio_idx)
        action_number = random.choice(ratios)
        action = shipyard_action(action_number, board, shipyard)
        if action != None:
            shipyard.next_action = action
    return me.next_actions
    '''

with open('submission.py', 'w') as f:
    f.write(my_agent)

In [ ]:
env.run(["./submission.py", "../input/kore2022bots/builder (26).py"])
env.render(mode="ipython", width=1000, height=800)

I made a q-leraning starter notebook, but I can't train this agent well.
I try 700 times in 11 hours by competing my best model which don't includes combat action. However, that agent's public score is 554.0.

<br />

There are several hypothetical reasons why it did not work.
1. The way the state is created is wrong.
2. I need to train more times such as 10000.
3. I need to let agent observe the board more using deep q-laning, etc.
4. In the end, the rule-based approach is stronger.